In [1]:
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from pandarallel import pandarallel
import math
random.seed(1234)

workdir = '../datasets/'
pandarallel.initialize(nb_workers=20)

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
click_f = pd.read_table(workdir + 'Gowalla/Gowalla_totalCheckins.txt', sep='\t', names=['uid', 'check_time', 'lat', 'lng', 'locid'])
trust_f = pd.read_table(workdir + 'Gowalla/Gowalla_edges.txt', sep='\t', names=['uid', 'fid'])
click_f['rating'] = 1
#calculate rating by geo voting
#calculate rating by geo voting

In [3]:
def rating_place(item_uni):
    #如果去的地方小于5个，那就按照54321的逆序评分给定分数
    #如果去的地方大于5个，按照去的地方个数平分评分
    RATING_COUNT = 5
    TRANSFER_POSITION = 2
    # 2是为了移动索引位置: [2 3 3 4 5 5 0 0] -> [1 1 2 3 3 4 5 5]
    rating = np.zeros_like(item_uni, dtype=int)
    
    if len(item_uni) < RATING_COUNT:
        last_item = item_uni[-1] + 1
        scores = 6
        for i in range(len(rating)):
            rating_ind = len(rating) - i - 1
            if item_uni[rating_ind] != last_item:
                scores = scores - 1
                rating[rating_ind] = scores
            elif item_uni[rating_ind] == last_item:
                rating[rating_ind] = rating[rating_ind + 1]
            last_item = item_uni[rating_ind]
    if len(item_uni) > RATING_COUNT:
        last_ind = 0
        for i in range(5):
            now_ind = math.floor(i * len(item_uni) / RATING_COUNT) + TRANSFER_POSITION
            rating[last_ind:now_ind] = i+1
            last_ind = now_ind
    return dict(zip(item_uni, rating))

In [154]:
example_locid = click_f.loc[click_f['uid']==0]['locid'].tolist()
(uniq, freq) = (np.unique(example_locid, return_counts=True))
# 定位id及次数
(item_uni, freq_uni) = np.unique(freq, return_counts=True)
# 次数频率统计
dict_rating = rating_place(item_uni)
print('count, rating\n去了count item 次的给rating item 分')
print(dict_rating)
dict_loctorating = dict(zip(uniq, [dict_rating[v] for v in freq]))

count, rating
去了count item 次的给rating item 分
{1: 1, 2: 1, 3: 2, 4: 3, 6: 3, 7: 4, 14: 5, 28: 5}


In [167]:
click_f.loc[click_f['uid']==0, 'rating'] = click_f.loc[click_f['uid']==0].parallel_apply(lambda r: dict_loctorating[r['locid']], axis=1)

In [4]:
unique_uid = np.unique(click_f['uid'].tolist())

for uid_ind in tqdm(range(len(unique_uid))):
    uid = unique_uid[uid_ind]
    example_locid = click_f.loc[click_f['uid']==uid]['locid'].tolist()
    (uniq, freq) = (np.unique(example_locid, return_counts=True))
    # 定位id及次数
    (item_uni, freq_uni) = np.unique(freq, return_counts=True)
    # 次数频率统计
    dict_rating = rating_place(item_uni)
    dict_loctorating = dict(zip(uniq, [dict_rating[v] for v in freq]))
    click_f.loc[click_f['uid']==uid, 'rating'] = click_f.loc[click_f['uid']==uid].parallel_apply(lambda r: dict_loctorating[r['locid']], axis=1)
    del dict_loctorating
click_f.to_csv('../datasets/Gowalla/Gowalla_rating.csv', index=False)

100%|██████████| 107092/107092 [8:29:54<00:00,  3.50it/s] 


In [11]:
click_f.to_csv('../datasets/Gowalla/Gowalla_rating.csv', index=False)

In [6]:
import zipfile, os
def file2zip(zip_file_name: str, file_names: list):
    with zipfile.ZipFile(zip_file_name, mode='w', compression=zipfile.ZIP_DEFLATED) as zf:
        for fn in file_names:
            parent_path, name = os.path.split(fn)
            zf.write(fn, arcname=name)

def zip2file(zip_file_name:str, extract_path: str, members=None, pwd=None):
    with zipfile.ZipFile(zip_file_name) as zf:
        zf.extractall(extract_path, members=members, pwd=pwd)

zip_name = '../datasets/Gowalla/Gowalla_rating.zip'
files = ['../datasets/Gowalla/Gowalla_rating.csv']
file2zip(zip_name, files)

In [135]:
click_list = []
trust_list = []

u_items_list = []
u_users_list = []
u_users_items_list = []
i_users_list = []

user_count = 0
item_count = 0
rate_count = 0
